In [2]:
import pandas as pd
import os
import sys
# Get the current directory
current_dir = os.getcwd()
# Move one directory up
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
# Add the parent directory to the system path
sys.path.insert(0, parent_dir)
import common_functions



In [ ]:
df = pd.read_excel(r"C:\Users\Wojciech\Documents\Python\MyOTAs\OTAs_Application\MyOTAs\working_files\Rimini 2024 TTG EXPO Italy.xlsx")
df

In [49]:
df_operators = pd.DataFrame()
sites = ['GYG', 'Viator', 'Musement', 'Headout']
for site in sites:
    file_manager = common_functions.FilePathManager(site, 'N/A')
    file_path_xlsx_operator = file_manager.get_file_paths()['file_path_xlsx_operator']
    print(file_path_xlsx_operator)
    temp_df = pd.read_excel(file_path_xlsx_operator)
    temp_df['Site'] = file_path_xlsx_operator.split('_')[-1].split('.')[0]
    df_operators = pd.concat([df_operators, temp_df])
    


G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Musement.xlsx
G:\.shortcut-targets-by-id\1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2\MyOTAs\Pliki firmowe\Operators_Headout.xlsx


In [67]:
import pandas as pd

# Assume df_operators is your DataFrame

# Step 1: Exclude unwanted entries and convert 'Reviews' to numeric
df_operators = df_operators[
    (df_operators['Operator'] != 'ToDo') & 
    (df_operators['Operator'] != 'InccorectURL')
]
df_operators['Reviews'] = pd.to_numeric(df_operators['Reviews'], errors='coerce')

# Step 2: Compute basic operator statistics
operator_stats = df_operators.groupby('Operator').agg(
    activity_count=('uid', 'nunique'),
    max_reviews=('Reviews', 'max'),
    min_reviews=('Reviews', 'min'),
    avg_reviews=('Reviews', 'mean')
).reset_index()

# Step 3: Calculate counts per 'Operator' and 'Site'
operator_site_counts = df_operators.groupby(['Operator', 'Site'])['uid'].nunique().reset_index()

# Step 4: Pivot the 'Site' column
operator_site_pivot = operator_site_counts.pivot(
    index='Operator', 
    columns='Site', 
    values='uid'
).reset_index()

# Step 4a: Fill NaN values with 0
operator_site_pivot.fillna(0, inplace=True)

# Step 4b: Ensure the site columns are integers
sites = ['GYG', 'Viator', 'Musement']
for site in sites:
    if site in operator_site_pivot.columns:
        operator_site_pivot[site] = operator_site_pivot[site].astype(int)
    else:
        # If the site is missing, add it with zeros
        operator_site_pivot[site] = 0

# Step 5: Merge with operator_stats
operator_stats = operator_stats.merge(
    operator_site_pivot[['Operator'] + sites], 
    on='Operator', 
    how='left'
)

# Step 5a: Fill any remaining NaN values with 0
operator_stats[sites] = operator_stats[sites].fillna(0).astype(int)

# Display the final DataFrame
operator_stats


,Operator,activity_count,max_reviews,min_reviews,avg_reviews,GYG,Viator,Musement
0,0,2,29.0,0.0,14.50,0,2,0
1,2,4,1.0,0.0,0.25,0,4,0
2,7,1,0.0,0.0,0.00,0,1,0
3,1388,1,0.0,0.0,0.00,0,1,0
4,1515,1,3.0,3.0,3.00,1,0,0
...,...,...,...,...,...,...,...,...
31755,å’Œç´™ãƒ©ãƒœTOKYO,1,1.0,1.0,1.00,0,1,0
31756,æ ªå¼ä¼šç¤¾ãƒãƒ³ã‚¯ãƒª,1,43.0,43.0,43.00,0,1,0
31757,é«˜æ©‹ã‚Šãˆå­,1,0.0,0.0,0.00,0,1,0
31758,ì turista Tours and trips,6,NaN,NaN,NaN,0,6,0


In [71]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Clean operator names
def clean_operator_name(name):
    suffixes = [
        'ltd', 'llc', 'inc', 'srl', 'kft', 'gmbh', 'société',
        'company', 'co', 'limited', 'plc', 'bv', 'ag'
    ]
    name = name.lower()
    for suffix in suffixes:
        name = re.sub(r'\b{}\b'.format(suffix), '', name)
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

df['Operator'] = df['Operator'].astype(str)
df['clean_operator'] = df['Operator'].apply(clean_operator_name)

operator_stats['Operator'] = operator_stats['Operator'].astype(str)
operator_stats['clean_operator'] = operator_stats['Operator'].apply(clean_operator_name)

operator_stats_names = operator_stats['clean_operator'].unique()

TOP_N = 2

def get_top_matches(name, choices, scorer=fuzz.token_sort_ratio, limit=TOP_N):
    matches = process.extract(name, choices, scorer=scorer, limit=limit)
    return matches

df['matches'] = df['clean_operator'].apply(
    lambda x: get_top_matches(x, operator_stats_names)
)

df_exploded = df.explode('matches')
df_exploded[['matched_operator', 'similarity_score', '_']] = pd.DataFrame(
    df_exploded['matches'].tolist(), index=df_exploded.index
)
df_exploded = df_exploded.drop(columns=['matches', '_'])

merged_df = pd.merge(
    df_exploded,
    operator_stats,
    how='left',
    left_on='matched_operator',
    right_on='clean_operator',
    suffixes=('_df', '_stats')
)

final_df = merged_df[[
    'Operator_df',
    'matched_operator',
    'similarity_score',
    'activity_count',
    'max_reviews',
    'min_reviews',
    'avg_reviews',
    'GYG',
    'Viator',
    'Musement'
]].rename(columns={
    'Operator_df': 'Original_Operator',
    'similarity_score': 'Similarity_Score'
})

final_df = final_df.sort_values(
    by=['Original_Operator', 'Similarity_Score'], 
    ascending=[True, False]
)



In [75]:
final_df['Similarity_Score'] = final_df['Similarity_Score']/100
final_df['Similarity_Score'] = final_df['Similarity_Score'].replace('.', ',')
final_df

,Original_Operator,matched_operator,Similarity_Score,activity_count,max_reviews,min_reviews,avg_reviews,GYG,Viator,Musement
0,1000 UT KFT,frhstck 3000,0.526316,2,30.0,29.0,29.500000,2,0,0
1,1000 UT KFT,1001 istanbul,0.500000,2,24.0,1.0,12.500000,0,2,0
2,181TRAVEL,4travel,0.750000,3,0.0,0.0,0.000000,0,3,0
3,181TRAVEL,atravel,0.750000,2,NaN,NaN,NaN,0,2,0
4,181TRAVEL,atravel,0.750000,2,4.0,4.0,4.000000,2,0,0
...,...,...,...,...,...,...,...,...,...,...
5166,ZÈTEMA PROGETTO CULTURA S.R.L.,theatri cultura napoletana,0.603774,1,NaN,NaN,NaN,0,1,0
3374,ÖBB FERROVIE AUSTRIACHE,astarita car service,0.604651,8,8.0,0.0,1.833333,0,8,0
3375,ÖBB FERROVIE AUSTRIACHE,lia beatrice ferrera,0.604651,2,0.0,0.0,0.000000,0,2,0
5169,ŽILINA TOURISM REGION,orion vision tourism,0.780488,2,3.0,0.0,1.500000,0,2,0


In [78]:
final_df.to_excel('OuputMatch.xlsx')